# Chargement du modèle GPT-2 entrainé

In [ ]:
!pip install --upgrade typing-extensions
!pip install gradio -q
!pip install keras_nlp -q


In [ ]:
import os
from tensorflow import keras
import keras_nlp
from google.colab import drive
import time

In [ ]:
os.environ["KERAS_BACKEND"] = "tensorflow"  # or "tensorflow" or "torch"

keras.mixed_precision.set_global_policy("mixed_float16")

preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,
)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en", preprocessor=preprocessor
)

/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


In [ ]:
drive.mount('/content/drive', force_remount=True)
checkpoint_path = "/content/drive/MyDrive/training_data_all_3/cp.ckpt"

gpt2_lm.load_weights(checkpoint_path)

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/task.py:47: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/task.py:47: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: 

# Chargement et configuration de gradio

In [ ]:
def generate_text(prompt):
    return gpt2_lm.generate(prompt, max_length=100)

In [ ]:

# CSS styles
css = """
body, html {
  height: 100%;
  margin: 0;
  font-family: 'Arial', sans-serif;
  background-color: #131722; /* Dark background color */
  color: #ffffff;
}

/* Container for the entire chat interface */
#chat-interface {
  display: flex;
  flex-direction: column;
  max-width: 80%; /* Ensure maximum width */
  height: 100vh;
  justify-content: space-between;
  margin: 0 auto; /* Center the chat interface */
}

/* Container for the chat messages */
#chat-messages {
  flex-grow: 1;
  overflow-y: auto;
  background: none;
  max-width: 100%; /* Ensure maximum width */
}

/* Styling for the chatbot bubble messages */
.gr-chatbot .chatbubble {
  max-width: 85%;
  margin-bottom: 12px;
  border-radius: 16px;
  padding: 12px 16px;
  position: relative;
  font-size: 1rem;
}

.gr-chatbot .chatbubble:before {
  content: '';
  position: absolute;
  width: 0;
  height: 0;
  border-style: solid;
}

/* Chatbot message bubble */
.gr-chatbot .bot .chatbubble {
  background-color: #2d3e55; /* Darker bubble background */
}

/* User message bubble */
.gr-chatbot .user .chatbubble {
  background-color: #4CAF50; /* Green bubble background */
}

/* Input area styling */
#input-area {
  display: flex;
  align-items: center;
  padding: 20px;
}

/* Text input field styling */
#input-area .gr-textbox {
  flex: 1;
  margin-right: 12px;
  padding: 12px 16px;
  border: 5px solid #627385;
  border-radius: 16px;
  font-size: 1rem;
}

/* Send button styling */
#input-area button {
  padding: 12px 20px;
  background-color: #4CAF50; /* Green button color */
  border: none;
  border-radius: 16px;
  cursor: pointer;
  font-size: 1rem;
  color: #fff;
}

/* Suggestion buttons styling */
.suggestion-btn {
  background-color: #2d3e55; /* Dark button color */
  color: #ffffff;
  padding: 10px 50px;
  margin: 5px;
  border: 2px solid #627385;
  border-radius: 20px;
  cursor: pointer;
  font-size: 14px;
  display: inline-block;
}

/* Suggestions container */
#suggestions {
  padding: 20px;
}

/* Style the avatar images if needed */
.gr-chatbot .gr-chatbot-avatar-image {
  border-radius: 50%;
}

/* Style for the chatbot avatar */
.gr-chatbot .bot .gr-chatbot-avatar-image {
  background-image: url('/content/drive/MyDrive/img/ALOQAS logo.png');
}

/* Style for the user avatar */
.gr-chatbot .user .gr-chatbot-avatar-image {
  background-image: url('/content/drive/MyDrive/img/pp discord copie.png');
}

/* Additional CSS for layout adjustments */
#header {
  display: flex;
  flex-direction: column;
  max-width: 100%; /* Ensure maximum width */
  gap: 20px;
  justify-content: center;
  align-items: center;
  margin: 0 auto; /* Center the chat interface */
}

#main-title {
  font-size: 2.5em;
  margin-bottom: 0.5em;
  color: #ffffff;
}

#sub-title {
  font-size: 1.5em;
  margin-bottom: 1em;
  color: #ffffff;
}

/* Adjust the chat interface to not grow beyond its container */
#chat-interface {
  flex: 1;
  overflow: auto; /* Add scrolling to the chat interface if needed */
}

.logo {
  width: 300px; /* Width of the logo */
  height: 300px; /* Height of the logo, should be equal to width for a perfect circle */
  background-image: url('https://github.com/LucasAguetai/ALOQAS/blob/main/Ressources/ALOQAS%20logo.png?raw=trueg');
  background-size: cover; /* Cover the entire area of the div without stretching */
  background-position: center; /* Center the background image within the div */
  border-radius: 50%; /* This will make it circular */
  display: inline-block; /* Allows the div to be inline with text and other inline elements */
  margin-bottom: 1em; /* Space below the logo */
}

#input-area > *{
  padding: 0px;
  border: 3px solid #627385;
}

#input-area > * > *{
  padding: 0px;
  background-color: #091E37;
}

#input-area > * * {
  border-radius: 0px !important;
}

.dark{
  --background-fill-primary: #091E37 !important;
}

.send{
  max-width: 10px;
  background-color: #627385 !important;
}
"""


In [ ]:
import gradio as gr

theme = gr.themes.Base(primary_hue="slate")

suggestion_text_1 = "What's the weather like today?"
suggestion_text_2 = "Can you provide stock market updates?"
suggestion_text_3 = "I need assistance with my account."

# Assuming generate_text is a function that generates a text response
def respond(message):
    # You will need to implement generate_response to create a response to the user's message.
    response = generate_text(message)
    return response

def respond(message):
    response = generate_text(message)
    return [["bot", response]]

def suggestion1():
    response = generate_text(suggestion_text_1)
    return [["user", suggestion_text_1], ["bot", response]]

def suggestion2():
    response = generate_text(suggestion_text_2)
    return [["user", suggestion_text_2], ["bot", response]]

def suggestion3():
    response = generate_text(suggestion_text_3)
    return [["user", suggestion_text_3], ["bot", response]]

with gr.Blocks(theme=theme, css=css) as demo:
    gr.Markdown("""
    <div id='header'>
      <h1 id='main-title'>ALOQAS</h1>
      <div class='logo'></div>
      <h2 id='sub-title'>How can I help you?</h2>
    </div>
    """)
    with gr.Column(elem_id="chat-interface"):
        chat = gr.Chatbot(elem_id="chat-messages", show_label=False)
        with gr.Row(elem_id="suggestions"):
            sugg1 = gr.Button(suggestion_text_1, elem_classes="suggestion-btn").click(
                suggestion1, outputs=chat
            )
            sugg2 = gr.Button(suggestion_text_2, elem_classes="suggestion-btn").click(
                suggestion2, outputs=chat
            )
            sugg3 = gr.Button(suggestion_text_3, elem_classes="suggestion-btn").click(
                suggestion3, outputs=chat
            )
        with gr.Row(elem_id="input-area"):
            text_input = gr.Textbox(placeholder="Write to ALOQAS...", show_label=False)
            send_button = gr.Button("Send", elem_classes="send")
            send_button.click(
                fn=respond,
                inputs=text_input,
                outputs=chat
            )

demo.launch(share=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>